# LangChain Fundamentals: Agents, Messages & Streaming

This notebook covers the core building blocks of LangChain 1.0:

1. **Building Agents** - The `create_agent()` API for building AI agents
2. **Messages** - The fundamental unit of context for LLM communication  
3. **Streaming** - Reducing latency by streaming responses

---

## Setup

First, let's install the required packages and set up our environment.

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langgraph

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Building Agents with `create_agent()`

In LangChain 1.0, the `create_agent()` function is the primary way to build AI agents. It provides a clean, declarative API that handles:

- Model selection and configuration
- Tool binding
- System prompts
- Runtime context (dependency injection)
- Middleware (like human-in-the-loop)

## 1.1 Your First Agent

Let's create a simple agent that can answer questions:

In [ ]:
from langchain.agents import create_agent

# Create a simple agent with a system prompt
agent = create_agent(
    model="openai:gpt-4o-mini",
    system_prompt="You are a helpful assistant that explains concepts clearly and concisely."
)

In [ ]:
# Invoke the agent with a question
result = agent.invoke({"messages": "What is LangChain?"})
print(result["messages"][-1].content)

## 1.2 Agent with Tools

Agents become powerful when they can use tools. Let's create a SQL agent that can query a database.

First, we'll set up a SQLite database and define a runtime context for dependency injection:

In [ ]:
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass

# Load the Chinook sample database (music store data)
db = SQLDatabase.from_uri("sqlite:///./assets-resources/Chinook.db")

# Define runtime context for dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite SELECT query and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

sql_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

In [ ]:
# Visualize the agent's ReAct loop
from IPython.display import Image, display

display(Image(sql_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Run a query - the agent will discover the schema and answer
question = "Which table has the largest number of entries?"

for step in sql_agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Another query - notice the agent self-corrects on errors
question = "Which genre on average has the longest tracks?"

for step in sql_agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Try Your Own Query

Modify the question below to explore the database:

In [ ]:
question = "What are the top 5 customers by total spending?"

for step in sql_agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

---

# Part 2: Messages - The Communication Backbone

Messages are the fundamental unit of context in LangChain. They represent the input and output of models, carrying both content and metadata needed to represent conversation state.

## Message Types

- **HumanMessage**: Input from users
- **AIMessage**: Responses from the model
- **SystemMessage**: Instructions for the model
- **ToolMessage**: Results from tool executions

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-4o-mini", 
    system_prompt="You are a helpful coding assistant."
)

In [ ]:
# Using HumanMessage explicitly
human_msg = HumanMessage("Explain what a decorator is in Python in one sentence.")

result = agent.invoke({"messages": [human_msg]})
print(result["messages"][-1].content)

In [ ]:
# Check the message types
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content[:80]}..." if len(msg.content) > 80 else f"{msg.type}: {msg.content}")

## 2.1 Alternative Message Formats

LangChain supports multiple ways to specify messages:

In [ ]:
# 1. Simple string (inferred as HumanMessage)
result = agent.invoke({"messages": "What is a list comprehension?"})
print("String input:", result["messages"][-1].content[:100], "...")

In [ ]:
# 2. Dictionary format
result = agent.invoke(
    {"messages": {"role": "user", "content": "What is a generator?"}}
)
print("Dict input:", result["messages"][-1].content[:100], "...")

## 2.2 Tool Messages in Action

When an agent uses tools, ToolMessages capture the results. Let's see this with a haiku-checking tool:

In [ ]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str) -> str:
    """Check if the given haiku text has exactly 3 lines.
    Returns validation result.
    """
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"Checking haiku with {len(lines)} lines")
    
    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct! This haiku has 3 lines."

haiku_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[check_haiku_lines],
    system_prompt="You are a poet who only writes Haiku. Always check your work before presenting it.",
)

In [ ]:
result = haiku_agent.invoke({"messages": "Write me a haiku about programming"})

# View all messages including tool calls
for msg in result["messages"]:
    msg.pretty_print()

## 2.3 Message Metadata

Messages contain rich metadata about model usage, tokens, and more:

In [ ]:
# Get the final AI message
final_message = result["messages"][-1]

print("Content:", final_message.content)
print("\nUsage metadata:", final_message.usage_metadata)
print("\nResponse metadata keys:", list(final_message.response_metadata.keys()))

---

# Part 3: Streaming - Reducing Latency

Streaming delivers information to users before the final result is ready. LangChain supports multiple streaming modes:

- **`values`**: Stream complete state updates after each node
- **`messages`**: Stream token-by-token (lowest latency)
- **`custom`**: Stream arbitrary data from tools

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-4o-mini",
    system_prompt="You are a helpful assistant.",
)

## 3.1 No Streaming (invoke)

In [ ]:
# Without streaming - waits for complete response
result = agent.invoke({"messages": "Tell me a short joke"})
print(result["messages"][-1].content)

## 3.2 Stream Mode: `values`

Streams the complete state after each step in the agent's execution:

In [ ]:
for step in agent.stream(
    {"messages": "Tell me a dad joke"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 3.3 Stream Mode: `messages` (Token-by-Token)

The lowest latency option - perfect for chatbots:

In [ ]:
for token, metadata in agent.stream(
    {"messages": "Write a short poem about Python programming."},
    stream_mode="messages",
):
    print(token.content, end="", flush=True)

## 3.4 Custom Streaming from Tools

You can stream custom data from within your tools using `get_stream_writer()`:

In [ ]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langgraph.config import get_stream_writer

@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    
    # Stream progress updates
    writer(f"Looking up weather data for {city}...")
    writer(f"Connecting to weather service...")
    writer(f"Data retrieved for {city}!")
    
    return f"It's sunny and 72°F in {city}!"

weather_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_weather],
)

In [ ]:
# Stream both values and custom data
for chunk in weather_agent.stream(
    {"messages": "What's the weather in San Francisco?"},
    stream_mode=["values", "custom"],
):
    stream_type, data = chunk
    if stream_type == "custom":
        print(f"[Progress] {data}")
    elif stream_type == "values":
        # Print only the final answer
        last_msg = data["messages"][-1]
        if hasattr(last_msg, 'content') and last_msg.content:
            print(f"[Answer] {last_msg.content}")

In [ ]:
# Stream only custom progress updates
print("Custom updates only:")
for chunk in weather_agent.stream(
    {"messages": "What's the weather in Tokyo?"},
    stream_mode=["custom"],
):
    print(f"  {chunk[1]}")

---

## Summary

In this notebook, we covered:

1. **`create_agent()`** - The unified API for building agents with:
   - Model selection (`model="openai:gpt-4o-mini"`)
   - Tool binding (`tools=[...]`)
   - System prompts
   - Runtime context for dependency injection

2. **Messages** - The communication backbone:
   - `HumanMessage`, `AIMessage`, `SystemMessage`, `ToolMessage`
   - Multiple input formats (string, dict, Message objects)
   - Rich metadata (usage, tokens, model info)

3. **Streaming** - Reducing latency:
   - `values`: Complete state after each step
   - `messages`: Token-by-token streaming
   - `custom`: Arbitrary data from tools

---

**Next:** [Notebook 2: Tools and Memory](./2.0-tools-and-memory.ipynb)